In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import sweetviz as sv
from matplotlib import pyplot as plt
import altair as alt
from altair import *
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from string import digits 
import spacy
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
#nltk.download('words')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Angel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
filepath = "D://archive"
df = pd.read_csv(filepath+'//goodreads_books.csv')
df.head(5)

,id,title,link,series,cover_link,author,author_link,rating_count,review_count,average_rating,five_star_ratings,four_star_ratings,three_star_ratings,two_star_ratings,one_star_ratings,number_of_pages,date_published,publisher,original_title,genre_and_votes,isbn,isbn13,asin,settings,characters,awards,amazon_redirect_link,worldcat_redirect_link,recommended_books,books_in_series,description
0,630104,Inner Circle,https://www.goodreads.com//book/show/630104.In...,(Private #5),https://i.gr-assets.com/images/S/compressed.ph...,"Kate Brian, Julian Peploe",https://www.goodreads.com/author/show/94091.Ka...,7597,196,4.03,3045,2323,1748,389,92,220.00,January 1st 2007,Simon Schuster Books for Young Readers,Inner Circle,"Young Adult 161, Mystery 45, Romance 32",1416950419,9781416950417,NaN,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"726458, 726458, 1537534, 3047848, 1651302, 304...","381489, 381501, 352428, 630103, 1783281, 17832...",Reed Brennan arrived at Easton Academy expecti...
1,9487,A Time to Embrace,https://www.goodreads.com//book/show/9487.A_Ti...,(Timeless Love #2),https://i.gr-assets.com/images/S/compressed.ph...,Karen Kingsbury,https://www.goodreads.com/author/show/3159984....,4179,177,4.35,2255,1290,518,93,23,400.00,October 29th 2006,Thomas Nelson,NaN,"Christian Fiction 114, Christian 45, Fiction 3...",1595542329,9781595542328,NaN,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"127352, 127352, 40642197, 127353, 127354, 3891...",115036,"Ideje az Ã¶lelÃ©snek TÃ¶rtÃ©net a remÃ©nyrÅl,..."
2,6050894,Take Two,https://www.goodreads.com//book/show/6050894-t...,(Above the Line #2),https://i.gr-assets.com/images/S/compressed.ph...,Karen Kingsbury,https://www.goodreads.com/author/show/3159984....,6288,218,4.23,3000,2020,1041,183,44,320.00,January 1st 2009,Zondervan,Take Two,"Christian Fiction 174, Christian 81, Fiction 58",NaN,NaN,NaN,"Bloomington, Indiana(United States)",NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"706250, 706250, 666481, 11942636, 706241, 1273...","4010795, 40792877, 7306261",Filmmakers Chase Ryan and Keith Ellison have c...
3,39030,Reliquary,https://www.goodreads.com//book/show/39030.Rel...,(Pendergast #2),https://i.gr-assets.com/images/S/compressed.ph...,"Douglas Preston, Lincoln Child",https://www.goodreads.com/author/show/12577.Do...,38382,1424,4.01,12711,15407,8511,1429,324,464.00,1997,Tor Books,Reliquary,"Thriller 626, Mystery 493, Horror 432, Fiction...",0765354950,9780765354952,NaN,"New York State(United States) New York City, N...","Aloysius X.L. Pendergast, Margo Green, William...",NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"39026, 39026, 11007668, 3986318, 51887330, 194...","67035, 39031, 39033, 136637, 136638, 30068, 39...",NaN
4,998,The Millionaire Next Door: The Surprising Secr...,https://www.goodreads.com//book/show/998.The_M...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,"Thomas J. Stanley, William D. Danko",https://www.goodreads.com/author/show/659.Thom...,72168,3217,4.04,27594,25219,14855,3414,1086,258.00,October 28th 1995,Gallery Books,The Millionaire Next Door: The Surprising Secr...,"Economics-Finance 1162, Nonfiction 910, Busine...",0671015206,9780671015206,NaN,NaN,NaN,Independent Publisher Book Award (IPPY) Nomine...,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"1052, 1052, 763362, 69571, 78427, 30186948, 10...",NaN,The incredible national bestseller that is cha...


In [4]:
df.columns

Index(['id', 'title', 'link', 'series', 'cover_link', 'author', 'author_link',
       'rating_count', 'review_count', 'average_rating', 'five_star_ratings',
       'four_star_ratings', 'three_star_ratings', 'two_star_ratings',
       'one_star_ratings', 'number_of_pages', 'date_published', 'publisher',
       'original_title', 'genre_and_votes', 'isbn', 'isbn13', 'asin',
       'settings', 'characters', 'awards', 'amazon_redirect_link',
       'worldcat_redirect_link', 'recommended_books', 'books_in_series',
       'description'],
      dtype='object')

In [5]:
df.shape

(52199, 31)

In [6]:
keep_cols = ['title', 'series', 'cover_link', 'author', 'rating_count', 'review_count', 
             'average_rating', 'five_star_ratings',
       'four_star_ratings', 'three_star_ratings', 'two_star_ratings',
       'one_star_ratings', 'number_of_pages', 'date_published', 'publisher',
       'genre_and_votes', 'settings', 'characters', 'awards', 'books_in_series',
       'description']

In [7]:
df = df[keep_cols]

In [8]:
df.describe()

,rating_count,review_count,average_rating,five_star_ratings,four_star_ratings,three_star_ratings,two_star_ratings,one_star_ratings,number_of_pages
count,52199.00,52199.00,52199.00,52199.00,52199.00,52199.00,52199.00,52199.00,49869.00
mean,18873.61,1012.98,4.02,7817.18,6250.78,3456.51,935.50,413.64,328.94
std,116397.83,4054.80,0.37,58763.73,34735.33,18249.30,5890.08,3843.36,252.79
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,340.00,31.00,3.82,117.00,109.00,63.00,15.00,6.00,210.00
50%,2295.00,163.00,4.03,810.00,765.00,452.00,107.00,36.00,304.00
75%,9297.50,622.00,4.23,3375.50,3190.50,1866.00,450.00,151.00,392.00
max,6801077.00,169511.00,5.00,4414877.00,1868421.00,980183.00,529060.00,537793.00,14777.00


In [9]:
# conda install -c conda-forge sweetviz
book_report = sv.analyze(df) #display the report
book_report.show_html('Best Books Goodreads.html')


Report Best Books Goodreads.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [9]:
df['series'] = df['series'].str.strip('()')
df

,title,series,cover_link,author,rating_count,review_count,average_rating,five_star_ratings,four_star_ratings,three_star_ratings,two_star_ratings,one_star_ratings,number_of_pages,date_published,publisher,genre_and_votes,settings,characters,awards,books_in_series,description
0,Inner Circle,Private #5,https://i.gr-assets.com/images/S/compressed.ph...,"Kate Brian, Julian Peploe",7597,196,4.03,3045,2323,1748,389,92,220.00,January 1st 2007,Simon Schuster Books for Young Readers,"Young Adult 161, Mystery 45, Romance 32",NaN,NaN,NaN,"381489, 381501, 352428, 630103, 1783281, 17832...",Reed Brennan arrived at Easton Academy expecti...
1,A Time to Embrace,Timeless Love #2,https://i.gr-assets.com/images/S/compressed.ph...,Karen Kingsbury,4179,177,4.35,2255,1290,518,93,23,400.00,October 29th 2006,Thomas Nelson,"Christian Fiction 114, Christian 45, Fiction 3...",NaN,NaN,NaN,115036,"Ideje az Ã¶lelÃ©snek TÃ¶rtÃ©net a remÃ©nyrÅl,..."
2,Take Two,Above the Line #2,https://i.gr-assets.com/images/S/compressed.ph...,Karen Kingsbury,6288,218,4.23,3000,2020,1041,183,44,320.00,January 1st 2009,Zondervan,"Christian Fiction 174, Christian 81, Fiction 58","Bloomington, Indiana(United States)",NaN,NaN,"4010795, 40792877, 7306261",Filmmakers Chase Ryan and Keith Ellison have c...
3,Reliquary,Pendergast #2,https://i.gr-assets.com/images/S/compressed.ph...,"Douglas Preston, Lincoln Child",38382,1424,4.01,12711,15407,8511,1429,324,464.00,1997,Tor Books,"Thriller 626, Mystery 493, Horror 432, Fiction...","New York State(United States) New York City, N...","Aloysius X.L. Pendergast, Margo Green, William...",NaN,"67035, 39031, 39033, 136637, 136638, 30068, 39...",NaN
4,The Millionaire Next Door: The Surprising Secr...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,"Thomas J. Stanley, William D. Danko",72168,3217,4.04,27594,25219,14855,3414,1086,258.00,October 28th 1995,Gallery Books,"Economics-Finance 1162, Nonfiction 910, Busine...",NaN,NaN,Independent Publisher Book Award (IPPY) Nomine...,NaN,The incredible national bestseller that is cha...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52194,The Stranger I Married,NaN,https://i.gr-assets.com/images/S/compressed.ph...,Sylvia Day,13295,805,3.87,4220,4733,3116,848,378,312.00,2007,Kensington Trade,"Romance-Historical Romance 335, Romance 312, H...","United Kingdom,1810","Gerard Faulkner, Isabel Pelham",NaN,NaN,"The unabridged, downloadable audiobook edition..."
52195,The Opposite of Loneliness: Essays and Stories,NaN,https://i.gr-assets.com/images/S/compressed.ph...,Marina Keegan,42609,4282,3.84,12461,15191,11202,3011,744,208.00,April 8th 2014,Scribner,"Nonfiction 1307, Short Stories 766, Writing-Es...",NaN,NaN,"Goodreads Choice Award for Nonfiction (2014), ...",NaN,An affecting and hope-filled posthumous collec...
52196,Sadako will leben,NaN,https://i.gr-assets.com/images/S/compressed.ph...,Karl Bruckner,1583,64,4.17,673,592,251,51,16,264.00,1961,Arena,"Fiction 13, Childrens 11, Historical-Historica...",NaN,Sadako Sasaki,"Austrian National Prize (1961), City of Vienna...",NaN,"6. August 1945, 8 Uhr 15 Minuten - die kleine ..."
52197,Confessions,Private #4,https://i.gr-assets.com/images/S/compressed.ph...,Kate Brian,8815,258,4.09,3760,2734,1780,435,106,240.00,April 24th 2007,Simon Schuster Books for Young Readers,"Young Adult 182, Mystery 52, Romance 34",NaN,NaN,NaN,"381489, 381501, 352428, 630104, 1783281, 17832...",Sometimes the truth hurts.... Reed Brennan...


cols = ['author', 'genre_and_votes', 'awards', 'books_in_series']
for col in cols:
    df[col] = [[idx for idx in x.split(',')] for x in df[col]]

In [10]:
#df['settings'] = df['settings'].str.findall(r'\(([^()]+)\)').str.join(',')

In [11]:
df.head()

,title,series,cover_link,author,rating_count,review_count,average_rating,five_star_ratings,four_star_ratings,three_star_ratings,two_star_ratings,one_star_ratings,number_of_pages,date_published,publisher,genre_and_votes,settings,characters,awards,books_in_series,description
0,Inner Circle,Private #5,https://i.gr-assets.com/images/S/compressed.ph...,"Kate Brian, Julian Peploe",7597,196,4.03,3045,2323,1748,389,92,220.00,January 1st 2007,Simon Schuster Books for Young Readers,"Young Adult 161, Mystery 45, Romance 32",NaN,NaN,NaN,"381489, 381501, 352428, 630103, 1783281, 17832...",Reed Brennan arrived at Easton Academy expecti...
1,A Time to Embrace,Timeless Love #2,https://i.gr-assets.com/images/S/compressed.ph...,Karen Kingsbury,4179,177,4.35,2255,1290,518,93,23,400.00,October 29th 2006,Thomas Nelson,"Christian Fiction 114, Christian 45, Fiction 3...",NaN,NaN,NaN,115036,"Ideje az Ã¶lelÃ©snek TÃ¶rtÃ©net a remÃ©nyrÅl,..."
2,Take Two,Above the Line #2,https://i.gr-assets.com/images/S/compressed.ph...,Karen Kingsbury,6288,218,4.23,3000,2020,1041,183,44,320.00,January 1st 2009,Zondervan,"Christian Fiction 174, Christian 81, Fiction 58","Bloomington, Indiana(United States)",NaN,NaN,"4010795, 40792877, 7306261",Filmmakers Chase Ryan and Keith Ellison have c...
3,Reliquary,Pendergast #2,https://i.gr-assets.com/images/S/compressed.ph...,"Douglas Preston, Lincoln Child",38382,1424,4.01,12711,15407,8511,1429,324,464.00,1997,Tor Books,"Thriller 626, Mystery 493, Horror 432, Fiction...","New York State(United States) New York City, N...","Aloysius X.L. Pendergast, Margo Green, William...",NaN,"67035, 39031, 39033, 136637, 136638, 30068, 39...",NaN
4,The Millionaire Next Door: The Surprising Secr...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,"Thomas J. Stanley, William D. Danko",72168,3217,4.04,27594,25219,14855,3414,1086,258.00,October 28th 1995,Gallery Books,"Economics-Finance 1162, Nonfiction 910, Busine...",NaN,NaN,Independent Publisher Book Award (IPPY) Nomine...,NaN,The incredible national bestseller that is cha...


We consider nan values as not having a sequel. 

In [12]:
df['books_in_series'] = [len([idx for idx in x.split(',')]) 
                         if pd.notna(x) else 0 for x in df['books_in_series']]
df['books_in_series']

0        10
1         1
2         3
3        10
4         0
         ..
52194     0
52195     0
52196     0
52197    10
52198     0
Name: books_in_series, Length: 52199, dtype: int64

In [13]:
# Cleaning authors

In [14]:
df['author'] = [[idx for idx in x.split(',')] for x in df['author']]
df['author']

0                         [Kate Brian,  Julian Peploe]
1                                    [Karen Kingsbury]
2                                    [Karen Kingsbury]
3                    [Douglas Preston,  Lincoln Child]
4               [Thomas J. Stanley,  William D. Danko]
                             ...                      
52194                                     [Sylvia Day]
52195                                  [Marina Keegan]
52196                                  [Karl Bruckner]
52197                                     [Kate Brian]
52198    [Sarah Palin,  Lynn Vincent,  Dewey Whetsell]
Name: author, Length: 52199, dtype: object

In [15]:
df.dropna(subset=['genre_and_votes'], inplace = True)
df.genre_and_votes = df.genre_and_votes.str.replace('\d+', '')
df.genre_and_votes = df.genre_and_votes.str.replace('user', '')
df.genre_and_votes = df.genre_and_votes.str.replace(' ', '')
df['genre_and_votes'] = [[idx for idx in x.split(',')] for x in df['genre_and_votes']]

In [16]:
df['genre_and_votes']

0                           [YoungAdult, Mystery, Romance]
1          [ChristianFiction, Christian, Fiction, Romance]
2                   [ChristianFiction, Christian, Fiction]
3                     [Thriller, Mystery, Horror, Fiction]
4        [Economics-Finance, Nonfiction, Business, Fina...
                               ...                        
52194    [Romance-HistoricalRomance, Romance, Historica...
52195    [Nonfiction, ShortStories, Writing-Essays, Aut...
52196    [Fiction, Childrens, Historical-HistoricalFict...
52197                       [YoungAdult, Mystery, Romance]
52198    [Biography, Nonfiction, Politics, Biography-Au...
Name: genre_and_votes, Length: 49359, dtype: object

In [26]:
df.loc['Fiction' in df['genre_and_votes']] 

KeyError: False

In [40]:
mask = ['Manga-Yaoi' in x for x in df['genre_and_votes']]
df[mask][['title', 'series', 'description']]

,title,series,description
2242,ä¸çä¸åæ ~å°éå¯ºå¾ã®å ´å3~,ä¸çä¸åæ (The World's Greatest First L...,æå¥½ãã ã£ãç·ã»é«éãä¸å¸ã¨ãã...
5164,"Sekaiichi Hatsukoi: A Boys Love Story, Volume 1",ä¸çä¸åæ (The World's Greatest First L...,Onodera Ritsu quits his job and transfers to t...
8402,Only the Flower Knows Vol. 1,è±ã®ã¿ãç¥ã [Hana no mizoshiru] #1,Youichi's life at university is going just fin...
9726,The Aristocrat and Desert Prince,NaN,"In his young life, Takeyuki has never really k..."
11363,"Only the Flower Knows, Band 2",è±ã®ã¿ãç¥ã [Hana no mizoshiru] #2,Zwischen Misaki und Arikawa scheint nach dem v...
13510,è±ã®ã¿ãç¥ã 3 [Hana no mizoshiru 3],è±ã®ã¿ãç¥ã [Hana no mizoshiru] #3,èª°ãã¨ä»ãåãã£ã¦ãã©ãããæã...
15484,CUT,NaN,Chiaki Sakaguchi is a highly sociable and popu...
16256,Jackass!,NaN,"When the panty hose go on, all bets are off be..."
18789,ã³ã³ããããã[Konbini-kun],NaN,To break away from his existence as a NEET (No...
19640,Don't Worry Mama,Don't Worry Mama Series #1,If you were stuck on a deserted island and you...


In [18]:
genres = df['genre_and_votes'].explode().value_counts().index.tolist()
genres

['Fiction',
 'Romance',
 'Fantasy',
 'YoungAdult',
 'Nonfiction',
 'Contemporary',
 'Historical-HistoricalFiction',
 'Mystery',
 'Classics',
 'Fantasy-Paranormal',
 'ScienceFiction',
 'Historical',
 'Childrens',
 'Thriller',
 'History',
 'Novels',
 'Adventure',
 'Biography',
 'Literature',
 'Humor',
 'Romance-ContemporaryRomance',
 'Mystery-Crime',
 'Horror',
 'Adult',
 'Philosophy',
 'ShortStories',
 'Romance-ParanormalRomance',
 'Audiobook',
 'Childrens-MiddleGrade',
 'Fantasy-UrbanFantasy',
 'WomensFiction-ChickLit',
 'Fantasy-Magic',
 'Autobiography-Memoir',
 'Paranormal-Vampires',
 'Christian',
 'SequentialArt-GraphicNovels',
 'Religion',
 'NewAdult',
 'SequentialArt-Comics',
 'Suspense',
 'Poetry',
 'RealisticFiction',
 'AdultFiction-Erotica',
 'ScienceFiction-Dystopia',
 'Politics',
 'ChristianFiction',
 'Science',
 'Psychology',
 'Romance-HistoricalRomance',
 'ScienceFictionFantasy',
 'SelfHelp',
 'War',
 'Thriller-MysteryThriller',
 'Spirituality',
 'Cultural-France',
 'LGBT',

In [19]:
df.sort_values(by=['rating_count'], ascending=False)

,title,series,cover_link,author,rating_count,review_count,average_rating,five_star_ratings,four_star_ratings,three_star_ratings,two_star_ratings,one_star_ratings,number_of_pages,date_published,publisher,genre_and_votes,settings,characters,awards,books_in_series,description
12665,Harry Potter and the Philosopher's Stone,Harry Potter #1,https://i.gr-assets.com/images/S/compressed.ph...,"[J.K. Rowling, Olly Moss]",6801077,108483,4.47,4414877,1557858,582217,133927,112198,345.00,June 26th 1997,Pottermore Publishing,"[Fantasy, Fiction, YoungAdult, Fantasy-Magic, ...","London, England,1991 Hogwarts School of Witchc...","Draco Malfoy, Ron Weasley, Petunia Dursley, Ve...",Mythopoeic Fantasy Award for Children's Litera...,6,Harry Potter has never even heard of Hogwarts ...
41555,Harry Potter and the Sorcerer's Stone,Harry Potter #1,https://i.gr-assets.com/images/S/compressed.ph...,"[J.K. Rowling, Mary GrandPrÃ©]",6774502,108035,4.47,4397134,1552070,580210,133456,111632,309.00,June 26th 1997,Scholastic Inc,"[Fantasy, Fiction, YoungAdult, Fantasy-Magic, ...","London, England,1991 Hogwarts School of Witchc...","Draco Malfoy, Ron Weasley, Petunia Dursley, Ve...",Mythopoeic Fantasy Award for Children's Litera...,6,Alternate cover edition of ISBN 9780439554930 ...
41514,The Hunger Games,The Hunger Games #1,https://i.gr-assets.com/images/S/compressed.ph...,[Suzanne Collins],6199172,169511,4.33,3349551,1868421,724660,166684,89856,374.00,September 14th 2008,Scholastic Press,"[YoungAdult, Fiction, ScienceFiction-Dystopia,...","District 12, Panem Capitol, Panem Panem(United...","Katniss Everdeen, Peeta Mellark, Cato (Hunger ...",Locus Award Nominee for Best Young Adult Book ...,2,In the ruins of a place once known as North Am...
41519,Twilight,The Twilight Saga #1,https://i.gr-assets.com/images/S/compressed.ph...,[Stephenie Meyer],4822006,102207,3.59,1695242,1079728,980183,529060,537793,501.00,October 5th 2005,"Little, Brown and Company","[YoungAdult, Fantasy, Romance, Paranormal-Vamp...","Forks, Washington(United States) Phoenix, Ariz...","Edward Cullen, Jacob Black, Laurent, Renee, Be...","Georgia Peach Book Award (2007), Buxtehuder Bu...",4,About three things I was absolutely positive. ...
41516,To Kill a Mockingbird,To Kill a Mockingbird,https://i.gr-assets.com/images/S/compressed.ph...,[Harper Lee],4353986,88833,4.28,2282987,1290816,557066,145436,77681,324.00,July 11th 1960,Harper Perennial Modern Classics,"[Classics, Fiction, Historical-HistoricalFicti...","Maycomb, Alabama,1933(United States) Alabama(U...","Scout Finch, Atticus Finch, Jem Finch, Arthur ...","Pulitzer Prize for Fiction (1961), Audie Award...",1,The unforgettable novel of a childhood in a sl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27865,Imp Empress: Falling in Love with Dragons: Aar...,The Society On Da Run,https://i.gr-assets.com/images/S/compressed.ph...,[L'Poni Baldwin],0,0,0.00,0,0,0,0,0,67.00,October 29th 2011,Geppetto Garten,[Romance-ParanormalRomance],Rome(Italy) Skyhouse Town(Ghana),NaN,NaN,0,Aaron Karrucci is the soon-to-be Emperor of An...
13241,You Win Even When You Lose: 50 Secrets And Tip...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,[D. Terry],0,0,0.00,0,0,0,0,0,17.00,January 14th 2013,BookTango,[SelfHelp],NaN,NaN,NaN,0,Losing weight is a daily battle for millions a...
15589,Drowned by Fear,Woodmere #3,https://i.gr-assets.com/images/S/compressed.ph...,[Sarah Richards],0,0,0.00,0,0,0,0,0,386.00,September 27th 2014,Createspace Independent Publishing Platform,[Mystery],NaN,NaN,NaN,0,Autumnâs senior year is filled with preparat...
35631,Pornography: The Other Side,NaN,https://i.gr-assets.com/images/S/compressed.ph...,[F.M. Christensen],0,0,0.00,0,0,0,0,0,188.00,May 1st 1990,Praeger Publishers,"[Gender, Pornography]",NaN,NaN,NaN,0,A unique and important book. I have never read...


In [20]:

from re import sub, match
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
import string

import nltk
words = set(nltk.corpus.words.words())
" ".join(w for w in nltk.wordpunct_tokenize(sent) \
         if w.lower() in words or not w.isalpha())

In [21]:
nltk.download('wordnet')
    import string
    

IndentationError: unexpected indent (<ipython-input-21-c085f8753aa3>, line 2)

In [50]:
sw = stopwords.words('english')
def full_cleaning(Category,frame,col):
    print('cleaning....')
    mask = [Category in x for x in df['genre_and_votes']]
    newframe = frame.copy()
    newframe = newframe[mask].astype(str)
    text = ' '.join(newframe[col][:])
    text = text.lower()
    text = sub(r'\[.*?\]', '', text)
    text = sub(r'([.!,?])', r' \1 ', text)
    text = sub(r'[^a-zA-Z.,!?]+', r' ', text)
    cleantext = [word for word in text if word not in sw]
    print(cleantext)
    #cleantext = clean_sent(cleantext)
    print('done!')
    return cleantext

In [51]:
def makingclouds(Category,frame,col,maximum,atitle):
    cloudtext=full_cleaning(Category,frame,col)
    print('preparing cloud')
    print('Cloud for',Category)
    wordcloud = WordCloud(max_font_size=40, max_words=maximum, 
                          stopwords=STOPWORDS,background_color="white",
                          collocations=False).generate(cloudtext)
    wordcloud.generate_from_frequencies
    plt.style.use("dark_background")
    plt.figure()
    plt.title(atitle, fontsize=30)#,fontweight="bold")
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")

In [52]:
interact(makingclouds, Category=genres, df=fixed(df), 
         col=fixed('description'),maximum=[100,500],
         atitle=fixed('Most used words in book summary'), 
         frame=fixed(df[['genre_and_votes','description']]))

interactive(children=(Dropdown(description='Category', options=('Fiction', 'Romance', 'Fantasy', 'YoungAdult',…

<function __main__.makingclouds(Category, frame, col, maximum, atitle)>

In [ ]:
sw = set(stopwords.words('english'))
def full_cleaning(Category,frame,col):
    print('cleaning....')
    mask = [Category in x for x in df['genre_and_votes']]
    newframe = frame.copy()
    newframe = newframe[mask].astype(str)
    text = ' '.join(newframe[col][:])
    text = text.lower()
    text = sub(r'\[.*?\]', '', text)
    text = sub(r'([.!,?])', r' \1 ', text)
    text = sub(r'[^a-zA-Z.,!?]+', r' ', text)
    #alphabet = list(string.ascii_lowercase)
    cleantext = [word for word in text if word not in sw]
    #cleantext = ' '.join(cleanlist)
    print(cleantext)
    #lemmatizer = WordNetLemmatizer()
    #cleantext = lemmatizer.lemmatize(cleantext)
    #cleantext = clean_sent(cleantext)
    print('done!')
    return cleantext